In [1]:
import threading, time, random

# ---------- Part 1: Concurrency Issues ----------
class BankAccount:
    def __init__(self, balance=1000):
        self.balance = balance
        self.lock = threading.Lock()

    def deposit_no_lock(self, amount):
        temp = self.balance
        time.sleep(random.uniform(0.01, 0.05))
        self.balance = temp + amount

    def withdraw_no_lock(self, amount):
        temp = self.balance
        time.sleep(random.uniform(0.01, 0.05))
        self.balance = temp - amount

    def deposit_safe(self, amount):
        with self.lock:
            temp = self.balance
            time.sleep(random.uniform(0.01, 0.05))
            self.balance = temp + amount

    def withdraw_safe(self, amount):
        with self.lock:
            temp = self.balance
            time.sleep(random.uniform(0.01, 0.05))
            self.balance = temp - amount


# Run threads without locks to see incorrect result
def simulate_concurrency_issue():
    acc = BankAccount(1000)
    threads = [
        threading.Thread(target=acc.deposit_no_lock, args=(100,)),
        threading.Thread(target=acc.withdraw_no_lock, args=(100,))
    ]
    for t in threads: t.start()
    for t in threads: t.join()
    print(f"[Part 1] Final balance (no locks): {acc.balance} (should be 1000)")

simulate_concurrency_issue()


[Part 1] Final balance (no locks): 1100 (should be 1000)


In [2]:
# ---------- Part 2: Deadlock Scenario ----------
db_lock = threading.Lock()
fraud_lock = threading.Lock()

def server_A():
    try:
        with db_lock:
            print("Server A: acquired DB, requesting Fraud Detection")
            time.sleep(0.1)
            with fraud_lock:
                print("Server A: acquired Fraud Detection")
    except Exception as e:
        print(f"Server A Error: {e}")

def server_B():
    try:
        with fraud_lock:
            print("Server B: acquired Fraud Detection, requesting DB")
            time.sleep(0.1)
            with db_lock:
                print("Server B: acquired DB")
    except Exception as e:
        print(f"Server B Error: {e}")

# Simulate potential deadlock
t1 = threading.Thread(target=server_A)
t2 = threading.Thread(target=server_B)
t1.start(); t2.start()
t1.join(timeout=1); t2.join(timeout=1)

print("[Part 2] Deadlock detected if threads hang due to circular wait.")


Server A: acquired DB, requesting Fraud Detection
Server B: acquired Fraud Detection, requesting DB
[Part 2] Deadlock detected if threads hang due to circular wait.


In [4]:
# ---------- Part 3: Wait-For Graph Analysis ----------
import networkx as nx

def wait_for_graph_example():
    G = nx.DiGraph()
    edges = [("A", "B"), ("B", "C"), ("C", "D"), ("D", "A")]
    G.add_edges_from(edges)
    print(f"[Part 3] Wait-For Graph Edges: {list(G.edges())}")
    cycle = list(nx.simple_cycles(G))
    print(f"[Part 3] Cycles detected: {cycle}")
    G.remove_node("D")
    print(f"[Part 3] After removing D, remaining edges: {list(G.edges())}")
    cycle_after = list(nx.simple_cycles(G))
    print(f"[Part 3] Deadlock persists after removing D? {bool(cycle_after)}")

wait_for_graph_example()


[Part 3] Wait-For Graph Edges: [('A', 'B'), ('B', 'C'), ('C', 'D'), ('D', 'A')]
[Part 3] Cycles detected: [['D', 'A', 'B', 'C']]
[Part 3] After removing D, remaining edges: [('A', 'B'), ('B', 'C')]
[Part 3] Deadlock persists after removing D? False


In [5]:
# ---------- Part 4: Mitchell & Merritt’s Algorithm ----------
def mitchell_merritt_probe_chain():
    probe_chain = [
        "PROBE(A, A→B)",
        "PROBE(A, B→C)",
        "PROBE(A, C→D)",
        "PROBE(A, D→A)"
    ]
    for step in probe_chain:
        print(f"[Part 4] {step}")
        time.sleep(0.1)
    print("[Part 4] Deadlock confirmed when probe returns to initiator (A).")

mitchell_merritt_probe_chain()


[Part 4] PROBE(A, A→B)
[Part 4] PROBE(A, B→C)
[Part 4] PROBE(A, C→D)
[Part 4] PROBE(A, D→A)
[Part 4] Deadlock confirmed when probe returns to initiator (A).


In [6]:
# ---------- Part 5: Deadlock Resolution ----------
def resolve_deadlock(priorities):
    try:
        lowest = min(priorities, key=priorities.get)
        print(f"[Part 5] Policy 1 (Kill lowest priority): Terminating Server {lowest}")
        alt1 = max(priorities, key=priorities.get)
        print(f"[Part 5] Policy 2 (Kill highest priority): Terminating Server {alt1}")
        chain = ["A", "B", "C", "D"]
        print(f"[Part 5] Policy 3 (Kill last in probe chain): Terminating Server {chain[-1]}")
    except Exception as e:
        print(f"Deadlock resolution error: {e}")
    finally:
        print("[Part 5] Freed resources returned to waiting servers.")

priorities = {"A": 3, "B": 2, "C": 1, "D": 4}
resolve_deadlock(priorities)


[Part 5] Policy 1 (Kill lowest priority): Terminating Server C
[Part 5] Policy 2 (Kill highest priority): Terminating Server D
[Part 5] Policy 3 (Kill last in probe chain): Terminating Server D
[Part 5] Freed resources returned to waiting servers.


In [7]:
# ---------- Part 6: Reflection ----------
def reflection_summary():
    print("[Part 6] Mitchell & Merritt’s algorithm is ideal for distributed systems.")
    print("It avoids centralized deadlock detection overhead and allows each node to participate in detection.")
    print("Concurrency control ensures atomicity and prevents ATM/freezing issues, improving reliability.")
    print("Deadlock resolution ensures high system availability and better customer experience.")

reflection_summary()


[Part 6] Mitchell & Merritt’s algorithm is ideal for distributed systems.
It avoids centralized deadlock detection overhead and allows each node to participate in detection.
Concurrency control ensures atomicity and prevents ATM/freezing issues, improving reliability.
Deadlock resolution ensures high system availability and better customer experience.
